## Recursive

In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit

def try_n_plus_or_N_plus(base_mol: Chem.Mol) -> str:
    """
    Given a molecule `base_mol` that presumably has exactly one
    quaternary nitrogen [N+]/n+, produce a base SMILES.
    
    Then create two candidate SMILES:
      (1) Replacing every '[N+]' with 'n+'  (the "aromatic" guess)
      (2) Replacing every 'n+'   with '[N+]' (the "aliphatic" guess)
    
    Attempt to sanitize each. If one passes, return that SMILES.
    If both pass, you can decide which to prefer (here we return the first that passes).
    If both fail, return the original base SMILES.
    """
    base_smi = Chem.MolToSmiles(base_mol, isomericSmiles=True)
    
    # Candidate 1: try "n+" in place of "[N+]"
    candidate1_smi = base_smi.replace("[N+]", "n+")
    candidate1_mol = Chem.MolFromSmiles(candidate1_smi)
    if candidate1_mol is not None:
        try:
            Chem.SanitizeMol(candidate1_mol)
            return Chem.MolToSmiles(candidate1_mol, isomericSmiles=True)
        except Exception:
            pass
    
    # Candidate 2: try "[N+]" in place of "n+"
    candidate2_smi = base_smi.replace("n+", "[N+]")
    candidate2_mol = Chem.MolFromSmiles(candidate2_smi)
    if candidate2_mol is not None:
        try:
            Chem.SanitizeMol(candidate2_mol)
            return Chem.MolToSmiles(candidate2_mol, isomericSmiles=True)
        except Exception:
            pass
    
    # If both fail, return the original base SMILES
    return base_smi


def has_min_carbon_chain(mol: Chem.Mol, n: int) -> bool:
    """
    Returns True if `mol` contains a (linear) chain of at least `n` carbons in a row.
    We construct an RDKit SMARTS that looks like: C-C-C-... (n times).
    """
    if mol is None:
        return False
    # e.g. for n=6: pattern = "C-C-C-C-C-C"
    pattern_str = "-".join(["C"] * n)
    pattern = Chem.MolFromSmarts(pattern_str)
    return mol.HasSubstructMatch(pattern)


def cut_once(
    smi: str, 
    min_carbons: int = 6
):
    """
    Attempt to cut the molecule 'smi' at exactly ONE [N+] substituent that:
      - contains no extra [N+],
      - and after removing that [N+], has at least `min_carbons` in a linear chain.

    Returns (sub_frag_smi, remainder_smi) if success,
            (None, None) if no valid cut is found.
    
    Uses the 'try_n_plus_or_N_plus' function to pick the correct N+ label in each piece.
    """
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return (None, None)

    # Find all [N+] atoms
    n_plus_indices = [
        a.GetIdx()
        for a in mol.GetAtoms()
        if a.GetAtomicNum() == 7 and a.GetFormalCharge() == 1
    ]
    if not n_plus_indices:
        return (None, None)
    
    for n_idx in n_plus_indices:
        neighbor_indices = [nbr.GetIdx() for nbr in mol.GetAtomWithIdx(n_idx).GetNeighbors()]
        
        for nb_idx in neighbor_indices:
            # BFS for that neighbor
            visited = set([nb_idx])
            queue = [nb_idx]
            while queue:
                curr = queue.pop(0)
                curr_atom = mol.GetAtomWithIdx(curr)
                for nbr in curr_atom.GetNeighbors():
                    nbr_idx = nbr.GetIdx()
                    if nbr_idx == n_idx:
                        continue
                    if nbr_idx not in visited:
                        visited.add(nbr_idx)
                        queue.append(nbr_idx)
            
            # Check if BFS substituent has another N+
            has_other_nplus = any(
                (mol.GetAtomWithIdx(a).GetAtomicNum() == 7 and 
                 mol.GetAtomWithIdx(a).GetFormalCharge() == 1)
                for a in visited
            )
            if has_other_nplus:
                continue
            
            # Build sub-mol with BFS + the chosen [N+]
            atoms_to_keep_sub = visited.union({n_idx})
            rwmol_sub = Chem.RWMol(mol)
            to_remove_sub = sorted(
                [a.GetIdx() for a in rwmol_sub.GetAtoms() 
                 if a.GetIdx() not in atoms_to_keep_sub],
                reverse=True
            )
            for x in to_remove_sub:
                rwmol_sub.RemoveAtom(x)
            sub_mol = rwmol_sub.GetMol()
            
            # Temporarily remove [N+] from sub_mol to check if it has an n-carbon chain
            n_plus_in_sub = [
                a.GetIdx() for a in sub_mol.GetAtoms()
                if a.GetAtomicNum() == 7 and a.GetFormalCharge() == 1
            ]
            if len(n_plus_in_sub) != 1:
                continue
            rwmol_for_chain = Chem.RWMol(sub_mol)
            rwmol_for_chain.RemoveAtom(n_plus_in_sub[0])
            sub_for_chain = rwmol_for_chain.GetMol()
            
            # Check if sub_for_chain has at least `min_carbons` in a row
            if not has_min_carbon_chain(sub_for_chain, min_carbons):
                continue
            
            # Build the remainder (everything except this BFS substituent minus the N index)
            atoms_to_keep_rem = set(range(mol.GetNumAtoms()))
            for vid in visited:
                if vid != n_idx:
                    atoms_to_keep_rem.discard(vid)
            
            rwmol_rem = Chem.RWMol(mol)
            to_remove_rem = sorted(
                [ix for ix in range(mol.GetNumAtoms()) 
                 if ix not in atoms_to_keep_rem],
                reverse=True
            )
            for x in to_remove_rem:
                rwmol_rem.RemoveAtom(x)
            remainder_mol = rwmol_rem.GetMol()
            
            # Fix each piece with "try_n_plus_or_N_plus"
            sub_frag_smi = try_n_plus_or_N_plus(sub_mol)
            remainder_smi = try_n_plus_or_N_plus(remainder_mol)
            
            return (sub_frag_smi, remainder_smi)

    return (None, None)


def cut_once_main(smi: str):
    """
    Tries to cut 'smi' using 6+ carbons first.
    If that fails, tries 4+ carbons.
    Returns (sub_frag_smi, remainder_smi) or (None, None).
    """
    subfrag_6, remainder_6 = cut_once(smi, min_carbons=6)
    if subfrag_6 is not None:
        return (subfrag_6, remainder_6)
    
    # Fall back to 4+ carbons
    return cut_once(smi, min_carbons=4)


def iterative_cut(smi: str, results=None):
    """
    Repeatedly apply `cut_once_main` until no more valid cuts remain.
    Each cut yields:
      - A sub-fragment (the BFS substituent + N+),
      - The remainder, which we keep cutting.
    Once we cannot cut further, we add that final 'smi' as a fragment.
    """
    if results is None:
        results = []
    
    subfrag, remainder = cut_once_main(smi)
    if subfrag is None:
        # No more cuts => add 'smi' as final (with correct labeling)
        mol = Chem.MolFromSmiles(smi)
        if mol is not None:
            final_smi = try_n_plus_or_N_plus(mol)
            results.append(final_smi)
        else:
            results.append(smi)
    else:
        # We made a cut => store the BFS subfragment
        results.append(subfrag)
        # Recursively cut the remainder
        iterative_cut(remainder, results)

    return results


# ------------------- EXAMPLE USAGE ------------------- #
if __name__ == "__main__":
    test_smi = "C[N+](C=C1)=CC=C1C2=CC=[N+](CCCCCCCCCCCCCCCC)C=C2.[Br-].[I-]"
    
    final_fragments = iterative_cut(test_smi)
    
    print("Fragments from iterative one-cut-at-a-time decomposition:")
    for i, frag in enumerate(final_fragments, 1):
        print(f"{i}. {frag}")


Fragments from iterative one-cut-at-a-time decomposition:
1. CCCCCCCCCCCCCCCC[n+]
2. C[n+]1ccc(C2=CC=[N+]C=C2)cc1.[Br-].[I-]


[06:52:24] non-ring atom 16 marked aromatic
[06:52:24] SMILES Parse Error: syntax error while parsing: CCCCCCCCCCCCCCCC[[N+]]
[06:52:24] SMILES Parse Error: Failed parsing SMILES 'CCCCCCCCCCCCCCCC[[N+]]' for input: 'CCCCCCCCCCCCCCCC[[N+]]'
[06:52:24] SMILES Parse Error: syntax error while parsing: C[n+]1ccc(C2=CC=n+C=C2)cc1.[Br-].[I-]
[06:52:24] SMILES Parse Error: Failed parsing SMILES 'C[n+]1ccc(C2=CC=n+C=C2)cc1.[Br-].[I-]' for input: 'C[n+]1ccc(C2=CC=n+C=C2)cc1.[Br-].[I-]'
[06:52:24] SMILES Parse Error: syntax error while parsing: C[[N+]]1ccc(C2=CC=[N+]C=C2)cc1.[Br-].[I-]
[06:52:24] SMILES Parse Error: Failed parsing SMILES 'C[[N+]]1ccc(C2=CC=[N+]C=C2)cc1.[Br-].[I-]' for input: 'C[[N+]]1ccc(C2=CC=[N+]C=C2)cc1.[Br-].[I-]'


In [56]:
tails = []
cores = []
error_tails = []

all_smi_list = []
with open('qac-0315.txt', "r") as f:
    for line in f:
        smi = line.strip()
        # Skip empty lines or whitespace-only lines
        if not smi:
            continue
        all_smi_list.append(smi)
        fragments = iterative_cut(smi)
        if len(fragments) == 0:
            print(smi)
            break
        core_smi = fragments[-1]
        
        mol_core = Chem.MolFromSmiles(core_smi)
        assert mol_core != None
        cores.append(core_smi)

        for tail_idx in range(len(fragments)-1):
            tail_smi = fragments[tail_idx]
            try:
                mol_tail = Chem.MolFromSmiles(tail_smi)
                assert mol_tail != None
                tails.append(tail_smi)
            except:
                tail_smi = tail_smi.replace("n", "N")
                mol_tail = Chem.MolFromSmiles(tail_smi)
                if mol_tail is not None:
                    tails.append(tail_smi)
                else:
                    error_tails.append(tail_smi)
        

[18:49:31] SMILES Parse Error: syntax error while parsing: CCCCCCCCCCCCn+
[18:49:31] SMILES Parse Error: Failed parsing SMILES 'CCCCCCCCCCCCn+' for input: 'CCCCCCCCCCCCn+'
[18:49:31] SMILES Parse Error: syntax error while parsing: Cn+(C)Cc1ccccc1.[Cl-]
[18:49:31] SMILES Parse Error: Failed parsing SMILES 'Cn+(C)Cc1ccccc1.[Cl-]' for input: 'Cn+(C)Cc1ccccc1.[Cl-]'
[18:49:31] SMILES Parse Error: syntax error while parsing: Cn+(C)Cc1ccccc1.[Cl-]
[18:49:31] SMILES Parse Error: Failed parsing SMILES 'Cn+(C)Cc1ccccc1.[Cl-]' for input: 'Cn+(C)Cc1ccccc1.[Cl-]'
[18:49:31] non-ring atom 16 marked aromatic
[18:49:31] SMILES Parse Error: syntax error while parsing: CCCCCCCCCCCCCCCC[[N+]]
[18:49:31] SMILES Parse Error: Failed parsing SMILES 'CCCCCCCCCCCCCCCC[[N+]]' for input: 'CCCCCCCCCCCCCCCC[[N+]]'
[18:49:31] SMILES Parse Error: syntax error while parsing: C1=CC=n+C=C1.[Cl-]
[18:49:31] SMILES Parse Error: Failed parsing SMILES 'C1=CC=n+C=C1.[Cl-]' for input: 'C1=CC=n+C=C1.[Cl-]'
[18:49:31] non-rin

In [58]:
# Writing to a file using a for loop
with open('0315-cores.txt', 'w', encoding='utf-8') as file:
    for line in set(cores):
        file.write(line + '\n')  # Adding a newline character



In [49]:
smi = all_smi_list[90]
fragments = iterative_cut(smi)

# Add them to our collections
print(smi)
print(fragments)

CCCCCCCCCCC[N+]1=CC(C2=CC=[N+](CCCCCCCCCCC)C=C2)=CC=C1.[Br-].[Br-]
['CCCCCCCCCCC[n+]', 'CCCCCCCCCCC[n+]', 'C1=C[N+]=CC(C2=CC=[N+]C=C2)=C1.[Br-].[Br-]']


[18:44:47] non-ring atom 11 marked aromatic
[18:44:47] SMILES Parse Error: syntax error while parsing: CCCCCCCCCCC[[N+]]
[18:44:47] SMILES Parse Error: Failed parsing SMILES 'CCCCCCCCCCC[[N+]]' for input: 'CCCCCCCCCCC[[N+]]'
[18:44:47] non-ring atom 11 marked aromatic
[18:44:47] SMILES Parse Error: syntax error while parsing: CCCCCCCCCCC[[N+]]
[18:44:47] SMILES Parse Error: Failed parsing SMILES 'CCCCCCCCCCC[[N+]]' for input: 'CCCCCCCCCCC[[N+]]'
[18:44:47] SMILES Parse Error: syntax error while parsing: C1=Cn+=CC(c2cc[n+]cc2)=C1.[Br-].[Br-]
[18:44:47] SMILES Parse Error: Failed parsing SMILES 'C1=Cn+=CC(c2cc[n+]cc2)=C1.[Br-].[Br-]' for input: 'C1=Cn+=CC(c2cc[n+]cc2)=C1.[Br-].[Br-]'
[18:44:47] SMILES Parse Error: syntax error while parsing: C1=C[N+]=CC(c2cc[[N+]]cc2)=C1.[Br-].[Br-]
[18:44:47] SMILES Parse Error: Failed parsing SMILES 'C1=C[N+]=CC(c2cc[[N+]]cc2)=C1.[Br-].[Br-]' for input: 'C1=C[N+]=CC(c2cc[[N+]]cc2)=C1.[Br-].[Br-]'
[18:44:47] SMILES Parse Error: syntax error while parsin

In [2]:
fragments = iterative_cut("O=C(C[N+]1=CC=C(CN(CC2=CC=[N+](C=C2)CC(OCCCCCC)=O)CC3=CC=[N+](C=C3)CC(OCCCCCC)=O)C=C1)OCCCCCC")
fragments

[06:53:08] non-ring atom 10 marked aromatic
[06:53:08] SMILES Parse Error: syntax error while parsing: CCCCCCOC(=O)C[[N+]]
[06:53:08] SMILES Parse Error: Failed parsing SMILES 'CCCCCCOC(=O)C[[N+]]' for input: 'CCCCCCOC(=O)C[[N+]]'
[06:53:08] non-ring atom 10 marked aromatic
[06:53:08] SMILES Parse Error: syntax error while parsing: CCCCCCOC(=O)C[[N+]]
[06:53:08] SMILES Parse Error: Failed parsing SMILES 'CCCCCCOC(=O)C[[N+]]' for input: 'CCCCCCOC(=O)C[[N+]]'
[06:53:08] SMILES Parse Error: syntax error while parsing: CCCCCCOC(=O)C[n+]1ccc(CN(CC2=CC=n+C=C2)Cc2cc[n+]cc2)cc1
[06:53:08] SMILES Parse Error: Failed parsing SMILES 'CCCCCCOC(=O)C[n+]1ccc(CN(CC2=CC=n+C=C2)Cc2cc[n+]cc2)cc1' for input: 'CCCCCCOC(=O)C[n+]1ccc(CN(CC2=CC=n+C=C2)Cc2cc[n+]cc2)cc1'
[06:53:08] SMILES Parse Error: syntax error while parsing: CCCCCCOC(=O)C[[N+]]1ccc(CN(CC2=CC=[N+]C=C2)Cc2cc[[N+]]cc2)cc1
[06:53:08] SMILES Parse Error: Failed parsing SMILES 'CCCCCCOC(=O)C[[N+]]1ccc(CN(CC2=CC=[N+]C=C2)Cc2cc[[N+]]cc2)cc1' for i

['CCCCCCOC(=O)C[n+]',
 'CCCCCCOC(=O)C[n+]',
 'CCCCCCOC(=O)C[n+]',
 'C1=CC(CN(CC2=CC=[N+]C=C2)CC2=CC=[N+]C=C2)=CC=[N+]1']